In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import permutations
import json, os
from collections import OrderedDict
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

from matplotlib import pyplot

In [2]:
pd.set_option('display.max_columns', None)

df = pd.read_csv('../../BPIC_Classification/processed_dataset/P_index_excat_15.csv')
df.head()

,Activity_0_01_HOOFD_010,Activity_0_01_HOOFD_011,Activity_0_01_HOOFD_020,Activity_0_01_HOOFD_030_2,Activity_0_01_HOOFD_065_0,Activity_0_01_HOOFD_065_2,Activity_0_01_HOOFD_080,Activity_0_01_HOOFD_100,Activity_0_01_HOOFD_101,Activity_0_01_HOOFD_110_0,Activity_0_01_HOOFD_190_2,Activity_0_01_HOOFD_195_2,Activity_0_01_HOOFD_450,Activity_0_01_HOOFD_460,Activity_0_01_HOOFD_490_3,Activity_0_01_HOOFD_490_5,Activity_0_01_HOOFD_491,Activity_0_01_HOOFD_516,Activity_0_02_DRZ_010,Activity_0_04_BPT_005,Activity_0_05_EIND_020,Activity_0_08_AWB45_051_2,Activity_0_08_OLO_100,Activity_0_11_AH_II_010,Activity_0_11_AH_II_036,Activity_0_11_AH_II_037_2,Activity_0_11_AH_II_040b,Activity_0_99_NOCODE_01,Activity_0_99_NOCODE_02,Activity_1_01_HOOFD_010,Activity_1_01_HOOFD_011,Activity_1_01_HOOFD_015,Activity_1_01_HOOFD_020,Activity_1_01_HOOFD_030_1,Activity_1_01_HOOFD_030_2,Activity_1_01_HOOFD_065_0,Activity_1_01_HOOFD_065_2,Activity_1_01_HOOFD_090,Activity_1_01_HOOFD_100,Activity_1_01_HOOFD_100_0,Activity_1_01_HOOFD_101,Activity_1_01_HOOFD_110_0,Activity_1_01_HOOFD_180,Activity_1_01_HOOFD_190_2,Activity_1_01_HOOFD_195_2,Activity_1_01_HOOFD_210_2,Activity_1_01_HOOFD_460,Activity_1_01_HOOFD_490_2,Activity_1_01_HOOFD_490_3,Activity_1_01_HOOFD_490_5a,Activity_1_01_HOOFD_491,Activity_1_01_HOOFD_510_2,Activity_1_01_HOOFD_510_2a,Activity_1_01_HOOFD_580_2,Activity_1_01_HOOFD_790,Activity_1_01_HOOFD_810,Activity_1_02_DRZ_010,Activity_1_02_DRZ_020_2,Activity_1_03_GBH_005,Activity_1_04_BPT_005,Activity_1_04_BPT_010,Activity_1_05_EIND_020,Activity_1_05_EIND_030_2,Activity_1_06_VD_060_2,Activity_1_08_AWB45_005,Activity_1_08_AWB45_020_2,Activity_1_08_AWB45_030,Activity_1_10_UOV_050_2,Activity_1_11_AH_II_037_2,Activity_1_11_AH_II_040b,Activity_1_11_AH_II_040c,Activity_1_12_AP_020_2,Activity_1_14_VRIJ_010,Activity_1_16_LGSV_010,Activity_1_99_NOCODE_01,Activity_1_99_NOCODE_02,Activity_2_01_HOOFD_010,Activity_2_01_HOOFD_011,Activity_2_01_HOOFD_012,Activity_2_01_HOOFD_015,Activity_2_01_HOOFD_020,Activity_2_01_HOOFD_030_1,Activity_2_01_HOOFD_030_2,Activity_2_01_HOOFD_040,Activity_2_01_HOOFD_065_0,Activity_2_01_HOOFD_065_1,Activity_2_01_HOOFD_065_2,Activity_2_01_HOOFD_090,Activity_2_01_HOOFD_100,Activity_2_01_HOOFD_100_0,Activity_2_01_HOOFD_101,Activity_2_01_HOOFD_110_0,Activity_2_01_HOOFD_180,Activity_2_01_HOOFD_190_2,Activity_2_01_HOOFD_195_2,Activity_2_01_HOOFD_196,Activity_2_01_HOOFD_330,Activity_2_01_HOOFD_350_2,Activity_2_01_HOOFD_410_2,Activity_2_01_HOOFD_460,Activity_2_01_HOOFD_490_3,Activity_2_01_HOOFD_490_5a,Activity_2_01_HOOFD_491,Activity_2_01_HOOFD_510_2,Activity_2_01_HOOFD_510_2a,Activity_2_01_HOOFD_580_2,Activity_2_01_HOOFD_790,Activity_2_02_DRZ_010,Activity_2_02_DRZ_010_2,Activity_2_02_DRZ_020_2,Activity_2_03_GBH_005,Activity_2_04_BPT_005,Activity_2_04_BPT_010,Activity_2_04_BPT_020,Activity_2_05_EIND_010,Activity_2_05_EIND_020,Activity_2_05_EIND_030_2,Activity_2_05_EIND_050,Activity_2_06_VD_030_2,Activity_2_06_VD_030_3,Activity_2_06_VD_030_5,Activity_2_07_OPS_010,Activity_2_08_AWB45_005,Activity_2_08_AWB45_020_1,Activity_2_08_AWB45_020_2,Activity_2_08_AWB45_040,Activity_2_10_UOV_030,Activity_2_10_UOV_035,Activity_2_10_UOV_050_2,Activity_2_10_UOV_050_2a,Activity_2_11_AH_II_037_2,Activity_2_12_AP_020_2,Activity_2_14_VRIJ_010,Activity_2_14_VRIJ_020_2,Activity_2_15_NGV_020_2,Activity_2_16_LGSV_010,Activity_2_99_NOCODE_02,Activity_3_01_HOOFD_010,Activity_3_01_HOOFD_011,Activity_3_01_HOOFD_012,Activity_3_01_HOOFD_015,Activity_3_01_HOOFD_020,Activity_3_01_HOOFD_030_1,Activity_3_01_HOOFD_030_2,Activity_3_01_HOOFD_040,Activity_3_01_HOOFD_050,Activity_3_01_HOOFD_060,Activity_3_01_HOOFD_065_0,Activity_3_01_HOOFD_065_2,Activity_3_01_HOOFD_090,Activity_3_01_HOOFD_100,Activity_3_01_HOOFD_100_0,Activity_3_01_HOOFD_101,Activity_3_01_HOOFD_110,Activity_3_01_HOOFD_110_0,Activity_3_01_HOOFD_180,Activity_3_01_HOOFD_190_2,Activity_3_01_HOOFD_195_2,Activity_3_01_HOOFD_200,Activity_3_01_HOOFD_210_2,Activity_3_01_HOOFD_350_0,Activity_3_01_HOOFD_450,Activity_3_01_HOOFD_460,Activity_

In [3]:
class AccuracyScore:

    def __init__(self, X_train, y_train, X_test, y_test):
        self.self = self
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def preprocessing(self):
        self['time:timestamp'] = pd.to_datetime(self['time:timestamp'], format='%Y/%m/%d %H:%M:%S.%f')
        self['time:timestamp'] = (self['time:timestamp'] - self['time:timestamp'].min()) / np.timedelta64(1,'M')
        
        return self
    
    def column_uniquify(self):
        cols = pd.Series(self.columns)
        dup_count = cols.value_counts()
        for dup in cols[cols.duplicated()].unique():
            cols[cols[cols == dup].index.values.tolist()] = [dup + str(i) for i in range(1, dup_count[dup]+1)]
        self.columns = cols
        return self.columns

    def data_split(self):
        X = df.drop('label', axis=1)
        y = df['label']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023, shuffle=True)
        
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        return X_train, y_train, X_test, y_test        
    
    def imbalance(self, X_train, y_train):
        sm = SMOTE(random_state=1)
        X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
        return X_train_res, y_train_res
    
    def DT(self, X_train, y_train, X_test, y_test):
        clf_dt = tree.DecisionTreeClassifier(max_depth=2, random_state=2023, max_features='log2')
        clf_dt = clf_dt.fit(X_train, y_train)
        predictions = clf_dt.predict(X_test)
        accuracy = clf_dt.score(X_test, y_test)
        print('DT Accuracy:', accuracy)
        print('DT AUC Score {}'.format(roc_auc_score(y_test, predictions)))

    def RF(self, X_train, y_train, X_test, y_test):
        clf_rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=2023)
        clf_rf = clf_rf.fit(X_train, y_train)
        predictions = clf_rf.predict(X_test)
        accuracy = clf_rf.score(X_test, y_test)
        print('RF Accuracy:', accuracy)
        print('RF AUC Score {}'.format(roc_auc_score(y_test, predictions)))
    
    def XGB(self, X_train, y_train, X_test, y_test):
        xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=2023)
        xgb.fit(X_train, y_train)
        xgb_pred = xgb.predict(X_test)

        accuracy = accuracy_score(y_test, xgb_pred)
        precision = precision_score(y_test, xgb_pred)
        recall = recall_score(y_test, xgb_pred)
        f1 = f1_score(y_test, xgb_pred)

        print('XGB Accuracy:', accuracy)
        print('XGB AUC Score {}'.format(roc_auc_score(y_test, xgb_pred)))
    
    def LGBM(self, X_train, y_train, X_test, y_test):
        clf_lgbm = LGBMClassifier(random_state=2023)
        clf_lgbm = clf_lgbm.fit(X_train, y_train)
        predictions = clf_lgbm.predict(X_test)
        accuracy = clf_lgbm.score(X_test, y_test)
        print('LGBM Accuracy:', accuracy)
        print('LGBM AUC Score {}'.format(roc_auc_score(y_test, predictions)))

In [4]:
df = AccuracyScore.preprocessing(df)
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.columns = AccuracyScore.column_uniquify(df)
X_train, y_train, X_test, y_test = AccuracyScore.data_split(df)
# X_train, y_train = AccuracyScore.imbalance(df, X_train, y_train)

In [5]:
AccuracyScore.DT(df, X_train, y_train, X_test, y_test)
AccuracyScore.RF(df, X_train, y_train, X_test, y_test)
AccuracyScore.XGB(df, X_train, y_train, X_test, y_test)
AccuracyScore.LGBM(df, X_train, y_train, X_test, y_test)

DT Accuracy: 0.7904689863842662
DT AUC Score 0.5017985611510791
RF Accuracy: 0.789712556732224
RF AUC Score 0.5
XGB Accuracy: 0.8373676248108926
XGB AUC Score 0.6133093525179856
LGBM Accuracy: 0.8055975794251135
LGBM AUC Score 0.565482097080956


In [12]:
df.shape

(4404, 1022)

In [1]:
import numpy as np

unique, counts = np.unique(y_train, return_counts=True)

print(np.asarray((unique, counts)).T)

NameError: name 'y_train' is not defined

In [14]:
y_test.value_counts()

1    1322
Name: label, dtype: int64